In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.gridspec import GridSpec
import scipy.io as sio
import warnings

import matplotlib.colors as mcolors
from scipy.io import loadmat
import seaborn as sns
import geopandas as gpd  # used for loading shapefile outline

from plot_config import params, SMALL_SIZE, MEDIUM_SIZE, LARGE_SIZE

warnings.filterwarnings("ignore")

# Apply author's rcParams
plt.rcParams.update(params)


In [ ]:
# Global constants (kept identical to current fig14.py visuals)

# Core Monsoon Zone polygon (same coordinates)
POLY_LON = np.array([86, 74, 74, 70, 70, 82, 82, 86, 86])
POLY_LAT = np.array([18, 18, 22, 22, 30, 30, 26, 26, 18])

# Line widths / styling (same values)
MAP_LW = 0.5
BOX_LW = 0.5
POLYGON_LW = 1.0

TICK_LENGTH = 2
TICK_WIDTH = 0.5
PANEL_LINEWIDTH = 0.5

# Box edge color (same: black)
BOX_COL = np.array([1, 1, 1]) * 0

# Discrete bounds & norm (same)
BOUNDS = np.linspace(0, 30, 11)
NORM = mcolors.BoundaryNorm(boundaries=BOUNDS, ncolors=256)

# Colormap (same as current code: seaborn YlOrBr)
CMAP = sns.color_palette("YlOrBr", as_cmap=True)

# Figure layout (same intent and numbers)
FIGSIZE = (8, 6)
GS_WIDTH_RATIOS = [1, 1, 0.03]
GS_WSPACE = 0.025
GS_LEFT = 0.05
GS_RIGHT = 0.94

# Manual colorbar axis placement relative to ax2 (same)
CBAR_X_GAP = 0.005
CBAR_WIDTH = 0.015
CBAR_HEIGHT_RATIO = 0.5  # matches your current code (0.5 of panel height)

# Axis ticks (same)
YTICKS = np.arange(10, 36, 5)
XTICKS = np.arange(70, 101, 5)


In [ ]:
# India outline loader

def get_india_outline():
    """
    Get India outline coordinates.

    Behavior is kept identical to the current script:
    1) Try a list of shapefile paths.
    2) If all fail, fall back to a simplified (india_lon, india_lat) pair.

    Returns
    -------
    list[tuple[list[float], list[float]]]
        A list of (lon_coords, lat_coords) boundary polylines.
    """
    shapefile_paths = [
        "../fig_data/ind_map_shapefile/india_shapefile.shp",
        "india_shapefile.shp",
    ]

    for path in shapefile_paths:
        try:
            india_gdf = gpd.read_file(path)
            boundaries = []

            for geom in india_gdf.geometry:
                # Single polygon
                if hasattr(geom, "exterior"):
                    coords = list(geom.exterior.coords)
                    lon_coords = [c[0] for c in coords]
                    lat_coords = [c[1] for c in coords]
                    boundaries.append((lon_coords, lat_coords))

                # Multi polygon
                elif hasattr(geom, "geoms"):
                    for sub_geom in geom.geoms:
                        if hasattr(sub_geom, "exterior"):
                            coords = list(sub_geom.exterior.coords)
                            lon_coords = [c[0] for c in coords]
                            lat_coords = [c[1] for c in coords]
                            boundaries.append((lon_coords, lat_coords))

            return boundaries
        except Exception:
            continue

    # Fall back (kept for parity with your original function)
    # NOTE: This fallback requires india_lon/india_lat to exist in scope;
    # if you truly need this fallback, define those arrays explicitly.
    return [(india_lon, india_lat)]


In [ ]:
# Geometry helpers

def point_in_polygon(x, y, poly_x, poly_y):
    """
    Check whether point (x, y) lies inside a polygon.

    Parameters
    ----------
    x, y : float
        Point coordinates.
    poly_x, poly_y : array-like
        Polygon vertex coordinates.

    Returns
    -------
    bool
        True if inside, False otherwise.
    """
    from matplotlib.path import Path
    polygon_path = Path(list(zip(poly_x, poly_y)))
    return polygon_path.contains_point((x, y))


def add_4deg_boxes_inside_polygon(ax, lon_centers, lat_centers, poly_x, poly_y):
    """
    Overlay 4°×4° box outlines for cells whose CENTER is inside the polygon.

    This matches your original logic exactly:
      - box size is 4 degrees
      - center at (xc, yc)
      - draw rectangle [xc-2, yc-2] with width/height 4
    """
    for xc in lon_centers.flatten():
        for yc in lat_centers.flatten():
            if point_in_polygon(xc, yc, poly_x, poly_y):
                rect = plt.Rectangle(
                    (xc - 2, yc - 2), 4, 4,
                    edgecolor=BOX_COL,
                    facecolor="none",
                    linewidth=BOX_LW
                )
                ax.add_patch(rect)


In [ ]:
# Axis formatting helper

def format_map_axes(ax, show_y_labels=True):
    """
    Apply axis formatting exactly as in the current script.

    - major ticks only
    - degree labels for N/E
    - equal aspect
    - fixed tick positions
    """
    ax.grid(False)
    ax.set_axisbelow(False)

    ax.tick_params("both", length=TICK_LENGTH, width=TICK_WIDTH, which="major")
    ax.tick_params(axis="x", which="minor", bottom=False, top=False)
    ax.tick_params(axis="y", which="minor", left=False, right=False)

    ax.set_yticks(YTICKS)
    if show_y_labels:
        yticklabels = [f"{int(y)}°N" for y in YTICKS]
        ax.set_yticklabels(yticklabels)
    else:
        ax.set_yticklabels([])

    ax.set_xticks(XTICKS)
    xticklabels = [f"{int(x)}°E" for x in XTICKS]
    ax.set_xticklabels(xticklabels)

    ax.tick_params(axis="both", which="major", labelsize=SMALL_SIZE,
                   length=TICK_LENGTH, width=TICK_WIDTH)

    for side in ["top", "right", "bottom", "left"]:
        ax.spines[side].set_linewidth(PANEL_LINEWIDTH)

    ax.set_aspect("equal", adjustable="box")


In [ ]:
# Cell 6 — Data loading (same file paths and variable names)

mat = loadmat("./mean_abs_subgrid_variability_fws_with_mok.mat")
matpt25 = loadmat("./pt25_mean_abs_subgrid_variability_fws_with_mok.mat")

diff_data = mat["mean_subgrid_variability_abs"]
diff_data_pt25 = matpt25["mean_subgrid_variability_abs"]

lon_1d = mat["lon_1deg"]     # 66.5 to 101.5 inclusive
lat_1d = mat["lat_1deg"]     # 6.5 to 39.5 inclusive

lonpt25 = matpt25["lon_pt25"]
latpt25 = matpt25["lat_pt25"]

lon_c = mat["lon_4deg"]
lat_c = mat["lat_4deg"]


In [ ]:
# Figure + axes + custom colorbar axis

fig = plt.figure(figsize=FIGSIZE)

gs = GridSpec(
    1, 3, figure=fig,
    width_ratios=GS_WIDTH_RATIOS,
    wspace=GS_WSPACE,
    left=GS_LEFT,
    right=GS_RIGHT,
)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])

# Create custom colorbar axis to the right of ax2, centered vertically
pos = ax2.get_position()
cbar_height = pos.height * CBAR_HEIGHT_RATIO
cbar_bottom = pos.y0 + (pos.height - cbar_height) / 2

cax = fig.add_axes([
    pos.x1 + CBAR_X_GAP,
    cbar_bottom,
    CBAR_WIDTH,
    cbar_height,
])


In [ ]:
# Panel (a): 1° grid

LON, LAT = np.meshgrid(lon_1d.flatten(), lat_1d.flatten())

im1 = ax1.pcolormesh(
    LON, LAT, diff_data.T,
    cmap=CMAP,
    norm=NORM,
    shading="auto",
    edgecolors="none",
    linewidth=0,
)

# India outline
india_boundaries = get_india_outline()
for india_lon_coords, india_lat_coords in india_boundaries:
    ax1.plot(india_lon_coords, india_lat_coords, color="black", linewidth=MAP_LW)

# Core monsoon polygon
poly = Polygon(list(zip(POLY_LON, POLY_LAT)), fill=False, edgecolor="black", linewidth=POLYGON_LW)
ax1.add_patch(poly)

# 4° boxes inside polygon
add_4deg_boxes_inside_polygon(ax1, lon_c, lat_c, POLY_LON, POLY_LAT)

# Axis limits (same)
ax1.set_xlim(lon_1d[0], lon_1d[-1])
ax1.set_ylim(lat_1d[0], lat_1d[-1])

# Format (keep y labels)
format_map_axes(ax1, show_y_labels=True)

# Panel label (same placement)
ax1.text(
    0.06, 0.98, "(a)",
    transform=ax1.transAxes,
    horizontalalignment="right",
    verticalalignment="top",
    color="black",
    fontsize=LARGE_SIZE,
    fontweight="normal",
)


In [ ]:
# Panel (b): 0.25° grid

LONpt25, LATpt25 = np.meshgrid(lonpt25.flatten(), latpt25.flatten())

im2 = ax2.pcolormesh(
    LONpt25, LATpt25, diff_data_pt25.T,
    cmap=CMAP,
    norm=NORM,
    shading="auto",
    edgecolors="none",
    linewidth=0,
)

# India outline
india_boundaries = get_india_outline()
for india_lon_coords, india_lat_coords in india_boundaries:
    ax2.plot(india_lon_coords, india_lat_coords, color="black", linewidth=MAP_LW)

# Core monsoon polygon
poly = Polygon(list(zip(POLY_LON, POLY_LAT)), fill=False, edgecolor="black", linewidth=POLYGON_LW)
ax2.add_patch(poly)

# 4° boxes inside polygon
add_4deg_boxes_inside_polygon(ax2, lon_c, lat_c, POLY_LON, POLY_LAT)

# Axis limits (same)
ax2.set_xlim(lon_1d[0], lon_1d[-1])
ax2.set_ylim(lat_1d[0], lat_1d[-1])

# Format (hide y labels)
format_map_axes(ax2, show_y_labels=False)

# Panel label
ax2.text(
    0.06, 0.98, "(b)",
    transform=ax2.transAxes,
    horizontalalignment="right",
    verticalalignment="top",
    color="black",
    fontsize=LARGE_SIZE,
    fontweight="normal",
)


In [ ]:
#  Colorbar + final save

cbar = plt.colorbar(im1, cax=cax, orientation="vertical", extend="max")
cbar.set_ticks(np.arange(0, 31, 6))
cbar.set_label("days", fontsize=MEDIUM_SIZE)
cbar.ax.tick_params(labelsize=SMALL_SIZE, length=2, width=1)
cbar.ax.minorticks_off()

plt.tight_layout()
plt.savefig("fig14.png", dpi=600, bbox_inches="tight")
plt.savefig("fig14.pdf", bbox_inches="tight")
plt.show()
